In [1]:
from glob import glob
import mne
import numpy as np
import pandas as pd

In [2]:
all_file_path = glob('data/*.edf')
teste_file_path = glob('data/seizures/*.edf')
print(len(all_file_path))
print(len(teste_file_path))

4
2


In [3]:
def read_data_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs=mne.make_fixed_length_epochs(data, duration=10, overlap=1)
    array=epochs.get_data()
    return array

def read_data_seizures(file_path):
    ref_file = open(file_path, 'r')
    array=epochs.get_data()
    return array

In [4]:
%%capture
data_epochs = [read_data_edf(i) for i in all_file_path]

In [5]:
########! Features extraction !#####
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np.argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x, axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmaxim(x), 
                           argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)),axis=-1)

In [6]:
data_array = np.vstack(data_epochs)

In [7]:
data_array.shape

(1596, 23, 2560)

In [8]:
features = []
for d in data_array:
    features.append(concatenate_features(d))
    
features_array = np.array(features)
features_array.shape

(1596, 276)

In [9]:
features_channel = []
MAX_CHANNELS = 23
for i in range(MAX_CHANNELS):
    array = features_array[:, i*12:(i+1)*12];
    features_channel.append(pd.DataFrame(array, columns=['Mean', 'Std', 'Ptp', 'Var', 'Minimo', 'Maxim', 'Arg Max', 'Arg Min', 'RMS', 'ABS DIFF', 'skewness', 'kurtosis']))

print('Info for channel 23: ')
features_channel[22]

Info for channel 23: 


,Mean,Std,Ptp,Var,Minimo,Maxim,Arg Max,Arg Min,RMS,ABS DIFF,skewness,kurtosis
0,0.090314,3.721824,3.206627,3.000922,1.162767,1.264415,5.786710,0.408149,2.602295,4.487221,3.842424,3.000922
1,12.771181,9.641350,5.539828,5.056912,12.993642,-0.036943,1.284505,0.858757,1.776437,8.681695,8.810274,5.056912
2,0.721598,8.640392,3.767408,1.969828,1.432256,0.811133,1.529032,0.593830,2.260915,6.781224,4.186863,1.969828
3,3.516893,9.109234,2.059038,10.979924,3.707853,0.688739,0.864438,5.914393,2.221584,7.196900,7.020388,10.979924
4,1.356837,5.328041,10.305580,1.233887,2.182984,0.792480,1.235111,1.270013,2.077118,13.130046,4.643348,1.233887
...,...,...,...,...,...,...,...,...,...,...,...,...
1591,0.457690,1.966667,1.646311,4.086531,1.305637,0.390626,6.349135,1.197140,0.608573,2.676397,2.162395,4.086531
1592,0.072534,21.821470,20.009509,5.613425,0.336918,0.108419,-0.113525,3.993927,8.120957,18.087594,22.009315,5.613425
1593,0.174822,3.534360,3.822112,8.800200,0.893302,0.351557,0.222093,0.573277,0.897243,2.707413,4.520891,8.800200
1594,0.977430,0.907855,2.418453,0.724176,5.137021,0.119911,6.710424,1.546738,2.109922,1.375174,2.098431,0.724176


In [10]:
print('Info for channel 0: ')
features_channel[0]

Info for channel 0: 


,Mean,Std,Ptp,Var,Minimo,Maxim,Arg Max,Arg Min,RMS,ABS DIFF,skewness,kurtosis
0,6.232756e-06,3.968335e-06,-3.282235e-07,-1.019223e-06,7.774734e-06,4.276501e-06,-3.529172e-06,2.978848e-07,5.887091e-06,-2.525362e-06,-7.317341e-06,-8.654882e-08
1,-3.172351e-08,3.149139e-07,3.806431e-07,-8.342241e-08,4.479354e-07,5.515627e-08,6.766285e-08,2.198528e-08,1.651275e-07,-2.372110e-07,-3.905805e-07,1.073958e-07
2,1.781872e-06,-2.512615e-07,-6.603173e-07,-7.508118e-07,7.974407e-07,1.901703e-07,-3.892110e-07,-4.789069e-07,1.363025e-06,7.716560e-08,-5.921740e-07,-4.995597e-07
3,2.131898e-06,-1.181364e-06,-6.441144e-07,-4.693231e-07,5.811912e-07,-3.866246e-07,-5.088253e-07,1.696957e-07,7.759505e-07,-2.196350e-07,1.171254e-07,1.194344e-07
4,4.458098e-07,2.824701e-07,-7.603957e-08,-1.747007e-07,3.361571e-07,1.742748e-07,1.046573e-08,-3.705838e-08,1.788149e-07,-7.616012e-08,-4.407808e-08,-8.833256e-10
...,...,...,...,...,...,...,...,...,...,...,...,...
1591,-6.499889e-07,1.791204e-07,2.112466e-07,1.596584e-08,-5.104883e-07,2.767760e-07,4.382419e-07,-4.498438e-07,-9.172804e-08,2.364566e-07,5.104591e-08,-3.419372e-08
1592,-2.071813e-07,-3.057387e-07,-1.686440e-07,3.450628e-07,-1.351770e-07,-2.641156e-07,-2.491515e-08,8.282607e-08,-9.830590e-08,-1.639648e-07,8.271716e-08,3.455449e-07
1593,1.586072e-06,-1.499760e-06,-2.487278e-07,-3.697330e-07,2.516618e-07,-8.109396e-07,-7.147408e-08,9.322562e-08,6.100184e-07,2.840509e-07,2.101447e-07,3.097448e-08
1594,-3.047318e-08,-3.780804e-07,-1.874030e-07,6.009834e-08,-3.119751e-07,-4.828091e-07,-3.205345e-08,3.096738e-07,-6.899988e-08,-1.519174e-07,3.542887e-07,-2.495720e-07
